# <span style='color:Blue'>P2 - Analyse des données des systèmes éducatifs</span>


## <span style='Thistle'>1. Introduction</span>

**Entreprise**
*****

Academy est une start-up spécialisée dans la formation et de cours en ligne pour un public diversifié(étudiants, professionnels )

**Mission**
*****
Dans le cadre du projet de la startup d'expansion internationale, il est primordiale de prendre de décisions et de formuler une stratégie d'implémentation dans de nouveaux pays. 

Suite à des échanges et reunions avec l'équipe métier, nous avons pu identifier quelques bésoins : 

- Valider la qualité de ce jeu de données (comporte-t-il beaucoup de données manquantes, dupliquées ?)
-  Décrire les informations contenues dans le jeu de données (nombre de colonnes ? nombre de lignes ?)

-  Sélectionner les informations qui semblent pertinentes pour répondre à la problématique (quelles sont les colonnes contenant des informations qui peuvent être utiles pour répondre à la problématique de l’entreprise ?)

- Déterminer des ordres de grandeurs des indicateurs statistiques classiques pour les différentes zones géographiques et pays du monde (moyenne/médiane/écart-type par pays et par continent ou bloc géographique)



L'analyse fonctionnelle technique  de besoins métiers nous amène à méner une analyse exploratoire de données afin d'extraire des informations pertinentes pour permettre à de prise de décisons. 

A partir des données de la Banque mondiale,  nous pouvons faire  une pré-analyse exploratoire afin de répondre aux différentes intérrogations métier: 

- Idenfier les pays à fort potentialité 
- Pour chacun de ces pays, quelle sera **l’évolution de ce potentiel** de clients ?
- Dans quels pays l'entreprise doit-elle **opérer en priorité** ?


**Démarche d'analyse**


- Analyse de la qualité données (**qualité**, **quantité**, **cohérence des données**)

- Créer un sous-groupe d'indicateurs  :
    - démographique 
    - éducatif 
    - économique 
    - numérique : 
  
- **Score** pour classer le potentiel des pays
- **Conclusions**


In [ ]:
# Importation des librairies


import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec 
import statsmodels as stat
from statsmodels.tsa.api import Holt
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

import geopandas as gpd
from math import pi
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_row',200)
pd.set_option('display.max_column',70)
pd.set_option('display.precision', 5)
# Verification de versions de librairies 
print('Version des librairies utilisées :')
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)
print('Geopandas     : ' + sns.__version__)
print('statsmodels   : ' + stat.__version__)







## <span style='Thistle'>2. Description et importation des jeux de données mis à disposition</span>

In [ ]:
# Importation des données en utilisant la fonction read_csv de pand

EdStatsCountry = pd.read_csv('/kaggle/input/dataset-edstats/Dataset_Edstats_csv/EdStatsCountry.csv')
EdStatsCountry_Series = pd.read_csv('/kaggle/input/dataset-edstats/Dataset_Edstats_csv/EdStatsCountry-Series.csv')
EdStatsData = pd.read_csv('/kaggle/input/dataset-edstats/Dataset_Edstats_csv/EdStatsData.csv')
EdStatsFootNote = pd.read_csv('/kaggle/input/dataset-edstats/Dataset_Edstats_csv/EdStatsFootNote.csv')
EdStatsSeries = pd.read_csv('/kaggle/input/dataset-edstats/Dataset_Edstats_csv/EdStatsSeries.csv')

In [ ]:
# Fonction permattant de calculer la somme des valeurs manquantes et afficher un diagramme en moustaches(boxplot)


def missng_values(df):

    total = df.isnull().sum()   # Calcul du total des valeurs manquantes
    percent = round(df.isnull().sum()/df.isnull().count()*100,2) # Calcul le pourcentage des valeurs 

    missing_data = pd.concat([total,percent],axis =1, keys=['Total','Ratio_of_NA(%)']) 
    type=pd.DataFrame(df[missing_data.index].dtypes, columns=['Types'])
    missing_data=pd.concat([missing_data,type],axis=1)
    missing_data=missing_data.sort_values('Total',ascending=False)
    missing_data.head(20)

    print( "le total des valeurs manquantes est", total.sum()) #Afficher la somme total des valeurs manquantes
    print( percent) #afficher le poucentage

    print(missing_data.head(20))
    print()
    print(set(missing_data['Types']))
    print()
    print("---Categorical columns---")
    
    print(missing_data[missing_data['Types']=="object"].index) # Colonnes ayant des variables qualitatives
    print()
    print("---Numerical columns----")
    print(missing_data[missing_data['Types'] !="object"].index)  # Colonnes ayant des variables quantitatives
   
    sns.set_style('whitegrid')
   


    plt.figure(figsize=(16,16)) #Réglage de la taille
    plt.title("Visualisation des valeurs manquantes") #titre 
    sns.heatmap(df.isnull(), cbar=False) #Affichage de la carte thermique

### <span style='PowderBlue'>2.1. Analyse du jeu de données EdStatsCountry.csv</span>

In [ ]:
# afficher les 3 premiers lignes enn utilisant la fonction head() de pandas 
EdStatsCountry.head(3)

In [ ]:
# afficher les 3 dernières lignes en utilisant la fonction tail() de pandas 
EdStatsCountry.tail(3)

In [ ]:
#afficher la dimmension du DataFrame  en utilisant la méthode shape

EdStatsCountry.shape

Le dataFrame EdStatsCountry a 241 lignes et 32 colonnes(variables) 

In [ ]:
#Affiche des noms des colnnes, le pourcentage de remplissages et le type de chaque colonne
EdStatsCountry.info()

Nous constatons que nous avons de types  primitives (float64, object)

In [ ]:
#affiche le type de chaque colonne

EdStatsCountry.dtypes

In [ ]:
#boucle qui permet de parcourir toutes les colonnes et ensuite de l'afficher

for col in EdStatsCountry.columns :
    print(col)

In [ ]:
#Affiche des valeurs distinctes de la colonne 'Income Group' pour identifier les possibilites de classement 

EdStatsCountry['Income Group'].unique()

Pour le classement de pays par niveau de revenu, on constate qu'il y'a de nan 

In [ ]:
# supprime la colonne 'Unnamed: 31' du dataframe EdStatsCountry

EdStatsCountry = EdStatsCountry.drop(['Unnamed: 31'], axis = 1)
EdStatsCountry

In [ ]:
EdStatsCountry.shape # Affiche les nouvelles valeurs de la dimension du DataFrame: 241 lignes, 31 colonnes

In [ ]:
# Appel de la fonction missng_values en incluant le dataframe 'EdStatsCountry' qui calcul et affiche les valeurs manquantes
    
missng_values(EdStatsCountry)

**National accounts reference year, Alternative conversion factor et Other groups ont beaucoup de valeurs manquantes dans le DataFrame**

In [ ]:
# Remplacer la région qui contient 'NaN' par 'Non Defini'

EdStatsCountry['Region'] = EdStatsCountry['Region'].replace(np.nan, 'Non Defini')
print('Régions :\n',EdStatsCountry['Region'].unique())

# Remplacer le groupe de revenus(Income Group) qui contient 'NaN' par 'Groupe de revenu Non Defini'
EdStatsCountry['Income Group'] = EdStatsCountry['Income Group'].replace(np.nan, 'Groupe de revenu Non Defini')
print('\nGroupes de revenus :\n',EdStatsCountry['Income Group'].unique())

Les valeurs manquantes Nan ont été remplacés par "Non Defini" et 'Groupe de revenu Non Defini'

In [ ]:
# Identification du Doublons dans le Dataframe par son identifiant unique(code pays)
doublons = EdStatsCountry[EdStatsCountry.duplicated(['Country Code'], keep=False)] # Renvoie une série booléenne indiquant les lignes en double
nb_doublons_code_pays = doublons.shape[0] 
print(f'Nombre de doublons sur le code pays : {nb_doublons_code_pays}') #affiche le nombre de doublons

In [ ]:
# Identification du Doublons dans le Dataframe par (short name)
doublons = EdStatsCountry[EdStatsCountry.duplicated(['Short Name'], keep=False)] # Renvoie une série booléenne indiquant les lignes en double
nb_doublons_code_pays = doublons.shape[0] 
print(f'Nombre de doublons sur le short name: {nb_doublons_code_pays}') #affiche le nombre de doublons

**Pays / Régions / Groupes de revenus**

In [ ]:
# Regroupement des pays par régions,par groupes de revenus, par regions et par groupes de revenus

fig=plt.figure(figsize=(15,10))
grid = plt.GridSpec(2, 2, wspace=0.8, hspace=0.3)  #Disposition en grille pour placer des sous-pareclles dans une figure 2 lignes 2 colonnes

# Premier Graphique sur la ligne 1, colonne 1 ------------------
plt.subplot(grid[0, 0])
# Regroupement des pays par régions
nb_pays_par_region=EdStatsCountry.groupby('Region')[['Short Name']].count().reset_index().sort_values(by='Short Name', ascending=False)
colors = ['Coral','Peru','HotPink','PowderBlue','Khaki','LightGreen','RoyalBlue','Cyan']
sns.barplot(nb_pays_par_region['Short Name'].sort_values(), nb_pays_par_region['Region'], palette=colors)
plt.xlabel('Nombre de pays', fontsize=12) # Titre des abscisses
plt.ylabel('Région', fontsize=12) #Titre des ordonnés
plt.title('Nombre de pays par Région', fontsize=14) #Titre du graphe

# Deuxieme Graphique sur la ligne 1, colonne 2 ------------------
plt.subplot(grid[0, 1])
# Regroupement des pays par groupe de revenus
nb_pays_par_groupe_revenu=EdStatsCountry.groupby('Income Group')[['Short Name']].count().reset_index().sort_values(by='Short Name', ascending=False)
colors = ['FireBrick','RoyalBlue','MediumPurple','MediumSeaGreen','SandyBrown','Khaki']
sns.barplot(nb_pays_par_groupe_revenu['Short Name'].sort_values(), nb_pays_par_groupe_revenu['Income Group'], palette=colors)
plt.xlabel('Nombre de pays', fontsize=12) # Titre des abscisses
plt.ylabel('Groupe de revenus', fontsize=12)  # Titre des ordonnés
plt.title('Nombre de pays par groupe de revenus', fontsize=14)#Titre du graphe

# Troisieme Graphique sur la ligne 2, sur les 2 colonnes ------------------
plt.subplot(grid[1, :])
# Regroupement des pays par groupe de revenus
colors = ['HotPink','PowderBlue','Peru','Coral','Khaki','LightGreen','RoyalBlue','Cyan']
sns.countplot(y='Income Group', hue='Region', data=EdStatsCountry, orient='h',
                   order=['High income: OECD', 'High income: nonOECD', 'Upper middle income', 'Lower middle income', 'Low income'],
                   palette=colors)
plt.xlabel('Nombre de pays', fontsize=12) # Titre des abscisses
plt.ylabel('Groupe de revenus', fontsize=12) # Titre des ordonnés
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=12)
plt.title('Nombre de pays par groupe de revenus par région', fontsize=14) #Titre du graphe
plt.grid(False)
plt.savefig("pie_char_EdStatsCountry")
plt.show()


Quelles interpretations peux tu faire de ce résultat 

***
**Bilan sur le jeu de données EdStatsCountry.csv**: </span>
***

-	Le dataset contient des informations géographiques sur les pays (**codes ISO-3166 alpha 2(2-alpha code ) et 3(code pays)**, code WB-2, les **noms des pays**, les regroupements pour 7 **régions,** par **groupe de revenus**…), des données économiques globales et des dates de référence des dernières études.
-	Le jeu de données contient **241 lignes et 32 colonnes** initialement.
-	Le nombre de **valeurs manquantes** totales est 2354 NaN pour 7712 observations et après suppression de la colonne Unnamed : 31 ,il reste 2113 NaN .
-	Il n'y a pas de doublons 
-	Aucune dobservation dupliquée sur les noms des pays et les codes pays. 
-	Toutes les régions du monde sont représentées.

***
*Notes :* à garder en mémoire si on utilise ce jeu de données
***

-	Certaines colonnes sont en float et contiennent des dates (années) : à transformer.
-	La colonne **Unnamed:31** a été supprimée.
-	Toutes les régions du monde sont représentées.

### <span style='PowderBlue'>2.2. Analyse du jeu de données EdStatsCountry-Series.csv</span>

In [ ]:
# afficher les 3 premiers lignes
EdStatsCountry_Series.head(3)

In [ ]:
#afficher la dimmension du DataFrame

EdStatsCountry_Series.shape

**Le dataFrame EdStatsCountry a 613 lignes et 4 colonnes(variables)**

In [ ]:
# Résumé des variables qualitatives
EdStatsCountry_Series.describe(exclude=[np.number])

In [ ]:
#Affiche des noms des colnnes, le pourcentage de remplissages et le type de chaque colonne
EdStatsCountry_Series.info()

In [ ]:
#affiche le type de chaque colonne

EdStatsCountry_Series.dtypes

In [ ]:
#boucle qui permet de parcourir toute les colonnes et ensuite de l'afficher

for col in EdStatsCountry_Series.columns :
    print(col)

In [ ]:
# supprime la colonne'Unnamed: 3'

EdStatsCountry_Series = EdStatsCountry_Series.drop(['Unnamed: 3'], axis = 1)
EdStatsCountry_Series

In [ ]:
# Appel de la fonction qui calcul les valeurs manquantes
    
missng_values(EdStatsCountry_Series)

In [ ]:
# Doublons sur les 2 colonnes CountryCode et SeriesCode
EdStatsCountry_Series[EdStatsCountry_Series.duplicated(keep=False)].shape[0]  # Renvoie une série booléenne indiquant les lignes en double

Il n'y a pas de doublons

**Pays présents dans EdStatsCountry.csv mais pas dans ce jeu de données EdStatsCountry-Series.csv en comparant le code pays**

In [ ]:
# Le nombre de code ISO aplha 3 ne correspond pas au code ISO alpha 3 du jeu de données EdStatsCountry.csv.

EdStatsCountry['Country Code'].nunique() == EdStatsCountry_Series['CountryCode'].nunique()

In [ ]:
# Recherche des noms des pays n'apparaissant pas dans le jeu de données EdStatsCountry-Series.csv
EdStatsCountry[~EdStatsCountry['Country Code'].isin(EdStatsCountry_Series['CountryCode'])][['Short Name']].sort_values(by = 'Short Name')

Les pays en plus dans le jeu de données EdStatsCountry.csv et ne sont pas le jeu de données EdStatsCountry_Series.csv sont :
- des noms de région, 
- ou des noms de regroupement de revenus, 
- ou certains regroupement de pays (European Union, Arab World..),
- et des pays (Curaçao, Nauru).

**Pays / Code indicateur / Description**

In [ ]:
# Regroupement des pays par codes indicateurs et par description

fig, ax = plt.subplots(1, 2, figsize=(15, 5))  #Fonction qui renvoie un objet figure et des axes
    
# Premier Graphique sur la ligne 1, colonne 1 ------------------

# Regroupement des pays par code indicateur
serie_code = EdStatsCountry_Series.groupby('SeriesCode')[['CountryCode']].count().reset_index().sort_values(by='CountryCode')['SeriesCode']
nb_pays_par_serie_code=EdStatsCountry_Series.groupby('SeriesCode')[['CountryCode']].count()['CountryCode'].sort_values()
ax1=ax[0]
ax1.barh(y = serie_code, width=nb_pays_par_serie_code, color='SteelBlue')
for i, v in enumerate(nb_pays_par_serie_code):
    ax1.text(v + 3, i - .22, str(v), color='SteelBlue', fontsize=10, fontweight='bold')
ax1.set_title('Nombre de pays par code indicateur', fontsize=12) 
ax1.set_xlabel('Nombre de pays', fontsize=10) #titre des abscisses
ax1.set_ylabel('Codes indicateurs', fontsize=10) #titre des ordonnés
ax1.tick_params(labelsize=10)
ax1.set_xlim([0, 250])
ax1.grid(False)

# Deuxieme Graphique sur la ligne 1, colonne 2 ------------------

# Regroupement des pays par description
description = EdStatsCountry_Series.groupby('DESCRIPTION')[['CountryCode']].count().reset_index().sort_values(by='CountryCode')['DESCRIPTION'].tail(10)
nb_pays_par_description=EdStatsCountry_Series.groupby('DESCRIPTION')[['CountryCode']].count()['CountryCode'].sort_values().tail(10)
ax2=ax[1]
ax2.barh(y = description, width=nb_pays_par_description, color='SteelBlue')
for i, v in enumerate(nb_pays_par_description):
    ax2.text(v + 3, i + .01, str(v), color='SteelBlue', fontsize=10, fontweight='bold')
ax2.set_title('Nombre de pays pour les tops 10 descriptions', fontsize=12) #titre du graphe
ax2.set_xlabel('Nombre de pays', fontsize=10) #titre des abscisses
ax2.set_ylabel('Sources des codes indicateurs', fontsize=10) #titre des ordonnés
ax2.tick_params(labelsize=10)
ax2.set_xlim([0, 180])
ax2.grid(False)

fig.tight_layout()
plt.savefig("pie_char_EdStatsCountry_Series")
plt.show()

***
**Bilan sur le jeu de données EdStatsCountry-Series.csv** : 
***

-  Le  dataset contient les **références des sources de certains indicateurs par pays** dans la colonne (Description) présents dans le jeu de données EdStatsCountry.csv.
-  Le jeu de données contient **613 lignes et 4 colonnes** initialement. 
-  Après suppresion de la colonne **Unnamed: 3** , il n'y a pas de **valeurs manquantes** ,les 3 premières colonnes ('Country Code', 'Serie Code' et 'Description') n’ont pas de valeurs manquantes.

***
*Notes : à garder en mémoire si on utilise ce jeu de données*
***

-  Les **pays en plus** (241 dans EdStatsCountry.csv et de 211 dans le jeu de données EdStatsCountry-Series.csv) sont des noms de région ou de regroupement de revenus ou certains regroupement de pays (European Union, Arab world..) et certains pays devenus autonomes (Curaçao).
-  La dernière variables **Unnamed: 3** a été supprimé.

### <span style='PowderBlue'>2.3.  Analyse du jeu de données EdStatsFootNote.csv</span>

In [ ]:
# Contenu des 3 premières lignes
EdStatsFootNote.head(3)

In [ ]:
# Contenu des 3 dernières lignes
EdStatsFootNote.tail(3)

In [ ]:
#afficher la dimmension du DataFrame

EdStatsFootNote.shape

**Le dataFrame EdStatsFootNote a 643638 lignes et 5 colonnes(variables)** 

In [ ]:
#Affiche des noms des colnnes, le pourcentage de remplissages et le type de chaque colonne
EdStatsFootNote.info()

In [ ]:
#affiche le type de chaque colonne

EdStatsFootNote.dtypes

In [ ]:
# supprime la colonne Unnamed

EdStatsFootNote = EdStatsFootNote.drop(['Unnamed: 4'], axis = 1)
EdStatsFootNote

In [ ]:
# Appel de la fonction qui calcul et affiche les valeurs manquantes
    
missng_values(EdStatsFootNote)

**Il n'y a pas de valeurs manquantes**

In [ ]:
# Doublon avec toutes les colonnes du dataframe
EdStatsFootNote[EdStatsFootNote.duplicated(keep=False)].shape[0]  # Renvoie une série booléenne indiquant les lignes en double

In [ ]:
# Doublons sur les 2 colonnes CountryCode et SeriesCode
doublons_deux = EdStatsFootNote[EdStatsFootNote.duplicated(['CountryCode', 'SeriesCode'], keep=False)]  # Renvoie une série booléenne indiquant les lignes en double
print(f'Nombre de doublons code pays/indicateur: {doublons_deux.shape[0]}.')


**Il y' a plusieurs doublons sur les colonnes CountryCode et SeriesCode du dataframe**

In [ ]:
# Doublons sur la colonne description
doublons_1 = EdStatsFootNote[EdStatsFootNote.duplicated(['DESCRIPTION'], keep=False)]
print(doublons_1.shape)

EdStatsFootNote['DESCRIPTION'].nunique()


**Il y'a plusieurs doublons sur la colonne description et les élements uniques sont au nombres de 9102**

In [ ]:
# Le nombre de code ISO alpha 3 ne correspond pas au code ISO alpha 3 du jeu de données EdStatsCountry.csv.
EdStatsCountry['Country Code'].nunique() == EdStatsFootNote['CountryCode'].nunique()

In [ ]:
# Recherche des noms des pays n'apparaissant pas dans le jeu de données EdStatsFootNote.csv
EdStatsCountry[~EdStatsCountry['Country Code'].isin(EdStatsFootNote['CountryCode'])][['Short Name']].sort_values(by = 'Short Name')
# ==> Les pays en plus dans le jeu de données EdStatsCountry sont des noms des pays devenus autonomes (Saint Martin, Groenland).

**Pays\Top 10 Code indicateur mis à jour - Top 10 des descriptions - Nombre de pays avec des mises à jour d'indicateur par années**

In [ ]:
# Regroupement des pays par code indicateur et des pays par description

plt.figure(figsize=(16, 10))
G = gridspec.GridSpec(2, 2)

# Graphique ligne 1, colonne 1 ------------------
ax1 = plt.subplot(G[0, 0])
# Top 10 du nombre de pays par code indicateur
serie_code = EdStatsFootNote.groupby('SeriesCode')[['CountryCode']].count().reset_index().sort_values(by='CountryCode')['SeriesCode'].tail(20)
nb_pays_par_serie_code=EdStatsFootNote.groupby('SeriesCode')[['CountryCode']].count()['CountryCode'].sort_values().tail(20)
ax1.barh(y = serie_code, width=nb_pays_par_serie_code,color='SteelBlue')
for i, v in enumerate(nb_pays_par_serie_code):
    ax1.text(v + 3, i - .2, str(v), color='SteelBlue', fontweight='bold', fontsize='12')
ax1.set_title('Top 10 du nombre de pays par code indicateur', fontsize=14) #titre du graphe
ax1.set_xlabel('Nombre de pays', fontsize='12') #titre des abscisses
ax1.set_ylabel('Code indicateur', fontsize='12')#titre des ordonnés
ax1.tick_params(labelsize=12)
ax1.grid(False)
ax1.set_xlim([0, 10800])

# Graphique ligne 1, colonne 2 ------------------
ax2 = plt.subplot(G[0, 1])
# Top 10 du nombre de pays par description
description = EdStatsFootNote.groupby('DESCRIPTION')[['CountryCode']].count().reset_index().sort_values(by='CountryCode')['DESCRIPTION'].tail(20)
nb_pays_par_description=EdStatsFootNote.groupby('DESCRIPTION')[['CountryCode']].count()['CountryCode'].sort_values().tail(20)
ax2.barh(y = description, width=nb_pays_par_description, color='SteelBlue')
for i, v in enumerate(nb_pays_par_description):
    ax2.text(v + 10, i - .3, str(v), color='SteelBlue', fontweight='bold', fontsize='12')
ax2.set_title('Top 10 du nombre de pays par description', fontsize='14') #titre du graphe
ax2.set_xlabel('Nombre de pays', fontsize='12')#titre des abscisses
ax2.set_ylabel('Description', fontsize='12') #titre des ordonnés
ax2.tick_params(labelsize=12)
ax2.grid(False)
ax2.set_xlim([0, 245000])

# Graphique ligne 2, sur les 2 colonnes ------------------
ax3 = plt.subplot(G[1, :])
# Pour plus de lisibilité, transformer par exemple 'YR2001' en '2001'
EdStatsFootNote['Annee']=[row[-4:] for row in EdStatsFootNote['Year']]
# Nombre de pays avec des mises à jour d'indicateur par années
ax3.set_title('Nombre de pays avec des mises à jour de codes indicateurs par années', size=15)#titre du graphe
ax3 = sns.barplot(y = 'CountryCode', x='Annee', data = EdStatsFootNote.groupby('Annee').count().reset_index(), color='SteelBlue')
ax3.set_xticklabels(labels = EdStatsFootNote.groupby('Annee').count().reset_index()['Annee'], rotation=90, size=12)
ax3.set_xlabel('Année de mise à jour')  #titre des abscisses
ax3.set_ylabel('Nombre de pays avec codes indicateurs mis à jour')#titre des ordonnés

plt.tight_layout()
plt.savefig("pie_char_EdStatsFootNote")
plt.show()

Le nombre de mise à jour des indicateurs semble régulier entre 1970 à 1990, suivi d'une légère augmentation entre 1991 et 1997, une accélération entre 1998 et 2013 puis de nouveau une chute des mises à jour qui sont presques nulles en 2017 et entre 2020 et 2050 (les années de prédictions).

***
**Bilan sur le jeu de données EdStatsFootNote.csv** : 
***

- Le jeu de données contient les **années de références de mises à jour des indicateurs par pays** et la **description des incertitudes, exceptions**, remarques sur les mises à jour.
- Le jeu de données contient **643638 lignes et 5 colonnes**.
- Nombre de **valeurs manquantes** totales : 643638 NaN pour 3218190 données (**20.0 %** : variable Unnamed: 4).
- Plusieurs dates ou description de mise à jour pour chacun des indicateurs possibles par pays.

***
*Notes : à garder en mémoire si on utilise ce jeu de données*
***

- Les pays non présents dans le jeu de données EdStatsFootNote.csv mais présents dans le jeu de données EdStatsCountry-Series.csv sont : 
    - Saint Martin,
    - Groenland.
- La variable **Unnamed: 4** est vide, elle pourra être supprimée si le jeu de donnée est utilisé.

### <span style='PowderBlue'>2.4. Analyse du jeu de données EdStatsSeries.csv</span>

In [ ]:
# Contenu des 3 premières lignes
EdStatsSeries.head(3)

In [ ]:
# Contenu des 3 dernières colonnes
EdStatsSeries.tail(3)

In [ ]:
# Nom des colonnes
EdStatsSeries.columns.to_list()

In [ ]:
#Affiche des noms des colnnes, le pourcentage de remplissages et le type de chaque colonne
EdStatsSeries.info()

In [ ]:
EdStatsSeries.shape # affiche les dimmensions du dataframe

**Le dataFrame EdStatsSeries a 3665 lignes et 21 colonnes(variables)**

In [ ]:
# supprime la colonne 'Unnamed: 20'

EdStatsSeries = EdStatsSeries.drop(['Unnamed: 20'], axis = 1)
EdStatsSeries

In [ ]:
# Appel de la fonction qui calcul les valeurs manquantes
    
missng_values(EdStatsSeries)

**Les colonnes 'Series Code','Topic','Indicator Name ','Short definition','Long definition'et 'sources' n'ont pas de valeurs manquantes**

In [ ]:
# Doublon avec toutes les colonnes du dataframe
EdStatsSeries[EdStatsSeries.duplicated(keep=False)].shape[0]  # Renvoie une série booléenne indiquant les lignes en double

In [ ]:
# Nombre de thèmes
unique=EdStatsSeries['Topic'].nunique()
print(f'{unique} thèmes regroupant les indicateurs')

In [ ]:
#Affiche le nombre des valeurs distinctes de  'Indicator Name'

indic = EdStatsSeries['Indicator Name'].nunique()
print(f'{indic} indicateurs')

In [ ]:
#Affiche des valeurs distinctes

EdStatsSeries['Topic'].unique()

**Codes indicateurs\ Top 10 thèmes \ Last 27 thèmes**

In [ ]:
# Regroupement des pays par code indicateur et des pays par thème

fig, ax = plt.subplots(1, 2, figsize=(15, 6))  

# Graphique ligne 1, colonne 1 ------------------
# Top 10 des thèmes les plus fréquents 
theme = EdStatsSeries.groupby('Topic')[['Series Code']].count().reset_index().sort_values(by='Series Code')['Topic'].tail(10)
nb_code_par_theme=EdStatsSeries.groupby('Topic')[['Series Code']].count()['Series Code'].sort_values().tail(10)
ax1=ax[0]
ax1.barh(y = theme, width=nb_code_par_theme, color='SteelBlue')
for i, v in enumerate(nb_code_par_theme):
    ax1.text(v + 3, i - .2, str(v), color='SteelBlue', fontweight='bold', fontsize=12)
ax1.set_title('Nombre d\'indicateurs pour les 10 thèmes les plus fréquents', fontsize=14) #titre du graphe
ax1.set_xlabel('Nombre de codes indicateurs', fontsize=12)#titre des abscisses
ax1.set_ylabel('Thème', fontsize=12) #titre des ordonnés
ax1.tick_params(labelsize=12)
ax1.grid(False)
ax1.set_xlim([0, 2000])

# Graphique ligne 1, colonne 2 ------------------
# last 27 thèmes les moins fréquents
theme_moins = EdStatsSeries.groupby('Topic')[['Series Code']].count().reset_index().sort_values(by='Series Code')['Topic'].head(27)
nb_code_par_theme_moins=EdStatsSeries.groupby('Topic')[['Series Code']].count()['Series Code'].sort_values().head(27)
ax2=ax[1]
ax2.barh(y = theme_moins, width=nb_code_par_theme_moins, color='SteelBlue')
for i, v in enumerate(nb_code_par_theme_moins):
    ax2.text(v + 1, i - .25, str(v), color='SteelBlue', fontweight='bold', fontsize=12)
ax2.set_title('Nombre d\'indicateurs pour les 27 thèmes les moins fréquents', fontsize=14) #titre du graphe
ax2.set_xlabel('Nombre de codes indicateurs', fontsize=12)  #titre des abscisses
ax2.set_ylabel('Thème', fontsize=12) #titre des ordonnés
ax2.tick_params(labelsize=12)
ax2.grid(False)
ax2.set_xlim([0, 50])#Définirles limites de la vue de l'axe X

fig.tight_layout()
plt.savefig("pie_char_EdStatsSeries")
plt.show()

***
**Bilan sur le jeu de données EdStatsSeries.csv** : 
***

- Le jeu de données permet de connaître le **thème des indicateurs**, les **descriptions longues et les sources**. Il donne des informations sur les indicateurs socio-éduco-économique classés en 37 thèmes.
- Le jeu de données contient **3665 lignes et 20 colonnes**.
- Nombre de **valeurs manquantes** totales 51538 NaN pour 76965 données.
- Aucun doublon.	

***
*Notes : à garder en mémoire si on utilise ce jeu de données*
***

- Le code indicateur, le Thème, le nom indicateur, la description longue et la source sont 100% complètes.
- Le reste des variables ne sont pas importantes et 6 variables sont complètement vides (donc à supprimer).

### <span style='PowderBlue'>2.5. Analyse du jeu de données EdStatsData.csv</span>

In [ ]:
# Affiche 3 lignes au hasard
EdStatsData.sample(3)

In [ ]:
# Contenu des 3 premières lignes
EdStatsData.head(3)

In [ ]:
# Contenu des 3 dernières colonnes
EdStatsData.tail(3)

In [ ]:
# Nom des colonnes
EdStatsData.columns

In [ ]:
#Affiche des noms des colnnes, le pourcentage de remplissages et le type de chaque colonne
EdStatsData.info()

In [ ]:
EdStatsData.shape #Affiche les dimensions du dataframe cad le nombre de ligne et de colonnes

**Le dataFrame EdStatsData a 886930 lignes et 70 colonnes(variables)**

In [ ]:
# supprime la colonne 'Unnamed: 69'

EdStatsData = EdStatsData.drop(['Unnamed: 69'], axis = 1)
EdStatsData

In [ ]:
# Appel de la fonction qui calcul et affiche les valeurs manquantes
    
missng_values(EdStatsData)

**Les colonnes 'Country Name','Country Code','Indicator Name','Indicator Code' n'ont pas de valeurs de manquantes**

In [ ]:
# Doublon avec toutes les colonnes du dataframe
EdStatsData[EdStatsData.duplicated(keep=False)].shape[0]


In [ ]:
# Doublons sur les 2 colonnes Country Code et Indicator Code
EdStatsData[EdStatsData.duplicated(['Country Code', 'Indicator Code'], keep=False)].shape[0]# Renvoie une série booléenne indiquant les lignes en double

**Il n'y a pas de doublons dans les colonnes Country Code et Indicator Code**

**Comparaison des pays présents dans EdStatsData.csv et EdStatsCountry.csv**

In [ ]:
# Le nombre de code ISO alpha 3 ne correspond pas au code ISO alpha 3 du jeu de données EdStatsCountry.csv.
EdStatsData['Country Code'].nunique() == EdStatsCountry['Country Code'].nunique()

In [ ]:
# Recherche du pays présent dans EdStatsData.csv mais pas dans EdStatsCountry.csv
df_pays_plus_Data=EdStatsData[~EdStatsData['Country Code'].isin(EdStatsCountry['Country Code'])][['Country Name']]
df_pays_plus_Data.drop_duplicates(inplace=True)
df_pays_plus_Data
# ==> Le pays en plus dans le jeu de données EdStatsData.csv est 'British Virgin Islands'.

**Comparaison des pays présents dans EdStatsData.csv et EdStatsCountries-Series.csv**

In [ ]:
# Le nombre de code ISO alpha 3 ne correspond pas au code ISO alpha 3 du jeu de données EdStatsCountry-Series.csv.
EdStatsData['Country Code'].nunique() == EdStatsCountry_Series['CountryCode'].nunique()

In [ ]:
# Recherche du pays présent dansEdStatsData.csv mais n'apparaissant pas dans le jeu de données EdStatsCountry-Series.csv
df_pays_plus_Data2=EdStatsData[~EdStatsData['Country Code'].isin(EdStatsCountry_Series['CountryCode'])][['Country Name']].sort_values(by = 'Country Name')
df_pays_plus_Data2.drop_duplicates(inplace=True)
df_pays_plus_Data2


Les pays en plus dans le jeu de données EdStatsData.csv sont : des noms de région, ou des noms de regroupement de revenus 
ou certains regroupement de pays (European Union, North America..) et certains pays devenus autonomes (Curaçao).

**Nombre de données par année / par décénnie**

In [ ]:
plt.figure(figsize=(15, 10))
G = gridspec.GridSpec(2, 2)

# Graphique ligne 1, sur les 2 colonnes ------------------
ax1 = plt.subplot(G[0, :])
# Répartition du nombre de données renseignées par années pour tous les indicateurs et tous les pays
present = EdStatsData.loc[:,'1970':'2100'].notnull().sum()
ax1 = present.plot(kind='bar', color='SteelBlue')
ax1.set_title('Répartition du nombre de données par années')
ax1.set_ylabel('Nombre de données')
ax1.grid(False)

# Graphique ligne 2, colonne 1 ------------------
ax2 = plt.subplot(G[1, 0])
# Regroupement du nombre de données par décennie, les années 70, 80...
decade=EdStatsData.loc[:,'1970':'2020'].copy().count()
ans=['1970s','1980s','1990s','2000s','2010s']
for i in range(5):
    j=i*10
    k=j+10
    decade[ans[i]]=decade[j:k,].sum()
# Affihage de la répartitions des données exploitables par décennie
width = 0.4   
ax2.bar(decade[ans].index, decade[ans].values, width, color='SteelBlue')
# ax2 = sns.barplot(y = decade[ans].values, x = decade[ans].index, color='SteelBlue')
ax2.set_xlabel('Décénnies')
ax2.set_ylabel('Nombre de données')
ax2.set_title('Nombre de données par décennie', size=12)
ax2.grid(False)

# Graphique ligne 2, colonne 2 ------------------
ax3 = plt.subplot(G[1, 1])
# Nombre de données totales
nb_tot=EdStatsData.shape[0]
# Nombre de données non nulles par année de la décénnie 2010
annees=['2010','2011','2012','2013','2015','2016','2017']
nb_nonnul=EdStatsData.copy()[annees].count()
df_2010s = pd.DataFrame({'annee':nb_nonnul.index, 'nb_nonnul':nb_nonnul.values})
# Taux de données non nulles par année de la décénnie 2010
df_2010s['%_nonnul']=round((df_2010s['nb_nonnul'])*100/nb_tot,2)
# Nombre de NaN par année de la décénnie 2010
df_2010s['%_nan']=round(100-df_2010s['%_nonnul'],2)
width = 0.35      
annes=df_2010s.index
taux_remplissage=df_2010s['%_nonnul']
taux_nan=df_2010s['%_nan']
ax3.bar(annees, taux_remplissage, width, label='Données', color='SteelBlue')
ax3.bar(annees, taux_nan, width, bottom=taux_remplissage, label='NaN', color='PeachPuff')
ax3.set_ylabel('%')
ax3.set_title('Taux de remplissage entre 2010 et 2017')
ax3.legend(bbox_to_anchor=(1.05, 1),loc = 2)
ax3.grid(False)

plt.tight_layout()
plt.savefig("pie_char_EdStatsData")
plt.show()

- Les années 2020 à 2100 sont des années de prédiction.
- Les années 2010 sont les années avec le plus de données significatives et parmi cette décennie l'année 2010 est la plus fournie avec seulement 27% de données.
- Les années 2016 et 2017, les plus proches de l'année courante (2021) contiennent moins de 2 % de variables et sont donc inexploitables.

***
**Bilan sur le jeu de données EdStatsData.csv :** 
***

- Ce jeu de données donne l'**évolution de plusieurs indicateurs sur une période de 1970 à 2017 pour tous les pays du monde** et certaines **macros régions du monde**, avec des **prédictions** pour les années 2020 - 2100, il semble donc très **intéressant pour notre problématique**.
- Le jeu de données contient **886930 lignes et 70 colonnes**.
- Nombre de **valeurs manquantes** totales : 53455179 NaN pour 62085100 données possibles.
- Aucun doublon.
- Les années 2020 à 2100 sont des années de prédictions non exploitables pour notre mission.
- Les variables des codes et noms indicateurs, des noms et des codes pays  sont 100% complètes.
- Très peu de données sont exploitables sur la période 1970-1997 et 2016-2017 (>94% de valeurs manquantes).
- L'année comportant le maximum de données est l'année 2010 mais avec 27% de données seulement.
- L'année la moins renseignée est l'année 2017 (99,98% de valeurs manquantes) donc inexploitable.
- L'année 2015 est exploitable.
- La variable unnamed a été supprimée

***
*Notes : à garder en mémoire si on utilise ce jeu de données*
***

- La variable 'Country Name' semble avoir des données autres que des noms de pays :
    - des noms de région, 
    - ou des noms de regroupement de revenus,
    - ou certains regroupement de pays (European Union, North America..)
  qui devront être nettoyées du jeu de données pour la pré-analyse.

### <span style='PowderBlue'>2.6. Conclusions sur les jeux de données</span>

- Ces jeux de données seront conservés pour la pré-analyse  :

    - ***EdStatsData.csv*** : qui contient la variable 'Country Name' et l'évolution des variables quantitatives des indicateurs par pays par années.
    - ***EdStatsCountry.csv*** : qui contient la = variable 'Short Name' et les régions et les groupes de revenus permettant de faire des régroupements.
    

*Notes :*    
***

- Le jeu de données **EdStatsSeries.csv** contient le **thème des indicateurs et leurs descriptions longues** qui pourra être utile lors de la sélection des indicateurs pertinents.    

## <span style='Thistle'>3. Sélection des indicateurs pertinents</span>

Notre entreprise Ccible :
- la population des 15-24 ans, 
- le nombre d'étudiants scolarisés(niveau lycée et universités).
- le niveau de vie des habitants comparable par pays
- les moyens de communication (accès à l'électricité, accès au réseau internet, possession d'un ordinateur ou tablette personnel).

En parcourant le site de la banque mondiale:

- les différents groupes d'indicateur à étudier sont :
    
    - **SE** : Social Education
    - **SP** : Social Population
    - **IT** : Infrastructure
    - **NY** : National Accounts, produits intérieurs et nationaux
    - **EG** : Energie

- les différents mots clés à rechercher :
    - **15** : pour la cible de la population des 15-19 ans
    - **20** : pour la cible de la population des 20-24 ans
    - **SEC** : pour les regroupements par lycéens
    - **TER** : pour les regroupements par étudiants de l'enseignement supérieur
    - **IT** : pour l'accès aux infrastructures techniques
    - **EG** : pour l'accès aux informations sur l'énergie

<span style='PowderBlue'>**Réduction temporelle(Année pertinents)**</span>

D'après la description des données de EdStastsData.csv, la décénnie 2010 est la plus fournie en données sur la période 2010-2015 (plus de 94% de NaN pour 2016-2017).

- **Plage temporelle** retenue : **2010-2015**.    

In [ ]:
annee_pertinente = EdStatsData.columns[44:-19].tolist() #Affiche une liste contenant la plage temporelle 2010-2015

annee_pertinente

**<span style='PowderBlue'>Réduction par seuil des indicateurs possibles</span>**

In [ ]:
# Creation d'un dataframe contenant tout les données essentielles

# Copie du dataframe EdStatsData à l'aide de la fonction copy()
df_Edstat_Total = EdStatsData.copy()

# Ajout du la long definition de l'indicateur
df_Edstat_Total = pd.merge(df_Edstat_Total , EdStatsSeries[['Series Code', 'Long definition']] , left_on = 'Indicator Code', right_on = 'Series Code' )
# Ajout de la région du pays
df_Edstat_Total = pd.merge( df_Edstat_Total, EdStatsCountry[['Country Code', 'Region']] , left_on = 'Country Code', right_on = 'Country Code' )
# Ajout du topic de l'indicateur
df_Edstat_Total = pd.merge( df_Edstat_Total , EdStatsSeries[['Series Code', 'Topic']] , left_on = 'Indicator Code', right_on = 'Series Code' )
#Ajout de Income group
df_Edstat_Total = pd.merge( df_Edstat_Total , EdStatsCountry[['Country Code', 'Income Group']] ,how = 'inner', left_on = 'Country Code', right_on = 'Country Code' )


In [ ]:
#Ajout de Income group
df_Edstat_Total = pd.merge( df_Edstat_Total , EdStatsCountry[['Country Code', 'Income Group']] ,how = 'inner', left_on = 'Country Code', right_on = 'Country Code' )

In [ ]:
#Suppression de la colonne : Series Code_y et 'Income Group_y''
del df_Edstat_Total['Series Code_y']
del df_Edstat_Total['Income Group_y']


In [ ]:
#Renomer les différentes colonnes car il ya des espace dans les noms de colonnes
df_Edstat_Total.rename(columns = {'Series Code_x': 'Series Code','Income Group_x' : 'Income Group'},inplace = True)

In [ ]:
#suppression de la colonne 'long definition', 'Series Code','Topic'
del df_Edstat_Total['Long definition']
del df_Edstat_Total['Series Code'] #suppresion du code indicateur
del df_Edstat_Total['Topic'] #suppression du thème

In [ ]:
#Enlever les dates non pertinentes et qui reprenent 94% de valeurs manquantes
date_a_supprime = df_Edstat_Total.loc[:,'1970':'2009']
df_Edstat_Total = df_Edstat_Total.drop(columns = date_a_supprime.columns) #suppression des dates avec 94% de valeurs manquantes


In [ ]:
#Reduction du DataFrame EdStatsData pour déterminer les indicateurs (référence année 2010, au moins à 40% de la ref)

#En effet le taux de remplissage max pour les années se trouve en 2010
list_columns = list(df_Edstat_Total.columns)
nb_Null = 0
reduction_seuil = df_Edstat_Total['2010'].notna().sum() * 0.4
print(reduction_seuil)
for col in  list_columns:
    if (df_Edstat_Total[col].notna().sum() < reduction_seuil) :
        df_Edstat_Total.drop(labels=[col], axis=1,inplace = True)
        print('colonne supprime :', col)
        print('---')
        nb_Null+=1
if nb_Null == 0 :
    print('Pas de colonne a supprimer')

In [ ]:
#Affichage des colonnes du jeu de données du dataframe total
df_Edstat_Total.columns

In [ ]:
#Création d'un fichier csv  de notre jeu de données df_Edstat_Total

df_Edstat_Total.to_csv('df_Edstat_Total.csv')


In [ ]:
#Affiche les 20 dernières colonnes tous les codes indicateurs qui sont égales à 'SE.SEC.ENRR'
df=df_Edstat_Total[df_Edstat_Total['Indicator Code']=='SE.SEC.ENRR']
df.tail(20) 

In [ ]:
#liste des indicteurs unique avec toutes les données renseignées
nbre_indicateur_unique= df_Edstat_Total['Indicator Name'].unique()
nbre_indicateur_unique.shape[0]

**Appliquer les filtres pour trouver des indicateurs pertinents**

In [ ]:
#Application du filtre pour trouver un indicateur démographique
# Cible : lycéens et étudiants de 15-24 ans
pd.set_option('max_colwidth', None)
liste_mot_cle_15=[row for row in df_Edstat_Total['Indicator Code'] if ('24') in row]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_15)][['Indicator Code','Indicator Name']];

Indicateur démographique retenu : **SP.POP.1524.TO.UN** qui représente la population total en ages de 15-24 ans

In [ ]:
# Cible : éducation lycée
pd.set_option('max_colwidth', None)
liste_mot_cle_sec=[row for row in df_Edstat_Total['Indicator Code'] if ('SEC') in row ]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_sec)][['Indicator Code','Indicator Name']];

Indicateur retenu : **SE.SEC.ENRR** taux brut de scolarisation dans le secondaire, pour les deux sexes (%) .

Nombre total d'inscriptions dans l'enseignement secondaire, quel que soit l'âge, exprimé en pourcentage de la population ayant l'âge officiel de fréquenter l'enseignement secondaire. Le TBS peut dépasser 100 % en raison de l'inclusion d'élèves trop âgés ou trop jeunes en raison d'une entrée précoce ou tardive à l'école et du redoublement.

In [ ]:
# Cible : éducation enseignement supérieur
liste_mot_cle_ter=[row for row in df_Edstat_Total['Indicator Code'] if ('TER') in row ]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_ter)][['Indicator Code','Indicator Name']];

Indicateur retenu : **SE.TER.ENRR** taux brut de scolarisation dans l'enseignement supérieur, pour les deux sexes (%).

Nombre total d'inscriptions dans l'enseignement supérieur (CITE 5 à 8), quel que soit l'âge, exprimé en pourcentage de la population totale du groupe d'âge de cinq ans suivant la fin de l'enseignement secondaire.

In [ ]:
# Cible : moyen de communication internet
liste_mot_cle_it=[row for row in df_Edstat_Total['Indicator Code'] if ('IT') in row ]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_it)][['Indicator Code','Indicator Name']];

Indicateur retenu : **IT.NET.USER.P2** Utilisateurs d'Internet (pour 100 personnes).

Les internautes sont des personnes qui ont utilisé l'internet (de n'importe quel endroit) au cours des trois derniers mois. L'internet peut être utilisé via un ordinateur, un téléphone portable, un assistant numérique personnel, une machine de jeux, la télévision numérique, etc.

*Note :* Aucun indicateur sur la détention d'un ordinateur personnel.

In [ ]:
# Cible : énergie, accès à l'éléctricité
liste_mot_cle_ene=[row for row in df_Edstat_Total['Indicator Code'] if ('EG') in row ]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_ene)][['Indicator Code','Indicator Name']];

Aucun indicateur permettant de connaître les informations sur l'accès à l'électricité.

In [ ]:
# Cible : économie
liste_mot_cle_eco=[row for row in df_Edstat_Total['Indicator Code'] if ('NY') in row ]
df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(liste_mot_cle_eco)][['Indicator Code','Indicator Name']];

Indicateur retenu : **NY.GDP.PCAP.PP.CD** RNB par habitant, PPA ($ international actuel).
    
Généralement, les pays à revenu élevé ont des niveaux de prix plus élevés, tandis que les pays à faible revenu ont des niveaux de prix plus bas (effet Balassa-Samuelson). Les comparaisons entre pays basées sur les taux de change du marché pour les composantes du PIB liées aux dépenses reflètent à la fois les différences de production économique (volumes) et de prix. Étant donné les différences de niveaux de prix, la taille des pays à revenu élevé est gonflée, tandis que la taille des pays à faible revenu est réduite dans la comparaison. Les comparaisons entre pays basées sur les PPA du PIB au niveau de ses composantes de dépenses reflètent uniquement les différences de production économique (volume), car les PPA contrôlent les différences de niveau de prix entre les pays. Ainsi, la comparaison reflète la taille réelle des pays. 

### Bilan sur les indicateurs pertinents

- démographique : 
    - **SP.POP.1524.TO.UN**
- économique : 
    - **NY.GNP.PCAP.PP.CD** 
- éducation : 
    - **SE.SEC.ENRR** 
    - **SE.TER.ENRR**
- numérique : 
    - **IT.NET.USER.P2** 

<span style='PowderBlue'>**Filtre avec les indicateurs pertinents**</span>

In [ ]:
# Liste des indicateurs pertinents à conserver
indicateurs_pertinent=['SP.POP.1524.TO.UN','SE.SEC.ENRR','SE.TER.ENRR','IT.NET.USER.P2','NY.GNP.PCAP.PP.CD']

# On ne conserve que les lignes donnant des informations sur les indicateurs pertinents
df_Edstat_Total=df_Edstat_Total[df_Edstat_Total['Indicator Code'].isin(indicateurs_pertinent)]
print(f'Nombre de lignes restantes : {df_Edstat_Total.shape[0]}')

<span style='PowderBlue'>**Suppression des pays hors regroupement et des groupes de revenus**</span>

In [ ]:
# Recherche des codes pays des regroupements de pays
df_pays_hors_norme=df_Edstat_Total[~df_Edstat_Total['Country Code'].isin(EdStatsCountry_Series['CountryCode'])][['Country Name','Country Code']].sort_values(by = 'Country Name')
df_pays_hors_norme.drop_duplicates(inplace=True) #supprime toutes les lignes contenant des doublons
pays_a_suppr=df_pays_hors_norme['Country Code'].to_list()   
df_Edstat_Total=df_Edstat_Total[~df_Edstat_Total['Country Code'].isin(pays_a_suppr)]
nombres_lignes=df_Edstat_Total.shape[0]
print(f'Nombre de lignes restantes : {nombres_lignes}')

In [ ]:
df_Edstat_Total.head(2) #Affiche les 2 premières lignes du dataframe

## <span style='Thistle'>4. Analyse des indicateurs pertinents</span>

In [ ]:
# Propagation des valeurs non nulles de 2010 vers 2015 pour travailler sur dernière valeur connue non nulle la plus récente en 2015
df_Edstat_Total_fillna=df_Edstat_Total.copy()
liste_des_dates=df_Edstat_Total_fillna.columns[4:-2].tolist()

#  Variables quantitatives
df_date_numeric = df_Edstat_Total_fillna[liste_des_dates] #dataframe comptant la plage temporelle 2010-2015

# Variables qualitatives
df_objet_categ = df_Edstat_Total_fillna.loc[:, ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Income Group', 'Region']]
df_date_numeric.fillna(method='ffill', axis=1, inplace=True)  # La méthode fillna transforme tous les types en object
df_Edstat_Total_fillna = pd.concat([df_objet_categ,df_date_numeric], axis=1) #concaténation des deux dataframes
nombre_de_pays=df_Edstat_Total_fillna['Country Name'].nunique()
print(f'Nombre de pays : {nombre_de_pays}')
# On ne conserve que la dernière année totalement remplie : 2015
df_Edstat_Total_2015=df_Edstat_Total_fillna[['Country Name','Country Code','Indicator Name','Indicator Code', 'Region', 
                       'Income Group','2015']]
# # Dernière colonne remplie renommée en 'Année Plus Récente'
df_Edstat_Total_2015.rename(columns={'2015': 'Année Plus Récente'}, inplace=True)
# # La méthode fillna transforme tous les types en object ==> retransformer la dernière colonne en numérique
df_Edstat_Total_2015=df_Edstat_Total_2015.astype({'Année Plus Récente': 'float64'}) #transforme en float64
df_Edstat_Total_2015

In [ ]:
#Graphiques statistiques sur les 5 indicateurs pertinents   
plt.figure(figsize = (15, 20))  

# ------------------------------------------------------------------------------------
# Indicateur démographique : SP.POP.1524.TO.UN
# Nombres de lycéens et étudiants de 15-24 ans par indicateur démographique
# ------------------------------------------------------------------------------------
df_pop=df_Edstat_Total_2015[df_Edstat_Total_2015['Indicator Code'] == 'SP.POP.1524.TO.UN']

plt.subplot(5,2,1) #Créer une figure et un ensemble de sous-parcelles
sns.boxplot( x=df_pop['Indicator Code'], y=df_pop['Année Plus Récente'], width=0.5, color='SteelBlue') #Crée une figure à moustaches à partir des colonnes DataFrame
plt.title('Indicateur démographique') #titre du graphe
plt.ylabel('Nombres lycéens et étudiants de 15-24 ans') #titre des ordonnés

plt.subplot(5,2,2) #Créer une figure et un ensemble de sous-parcelles
sns.histplot(df_pop['Année Plus Récente'], kde = True, color='SteelBlue') #Dessine un histogramme à partir des colonnes du DataFrame
plt.title('Indicateur démographique')
plt.xlabel('SP.POP.1524.TO.UN')

# ------------------------------------------------------------------------------------
# Indicateur économique : NY.GNP.PCAP.PP.CD
# Revenu par tête par indicateur économique
# ------------------------------------------------------------------------------------
df_eco=df_Edstat_Total_2015[df_Edstat_Total_2015['Indicator Code'] == 'NY.GNP.PCAP.PP.CD']

plt.subplot(5,2,3) #Créer une figure et un ensemble de sous-parcelles
sns.boxplot( x=df_eco['Indicator Code'], y=df_eco['Année Plus Récente'], width=0.5, color='SteelBlue') #Crée une figure à moustaches à partir des colonnes DataFrame
plt.title('Indicateur économique')
plt.ylabel('Revenu par tête(RNB/habitant), PPA ($)')

plt.subplot(5,2,4) #Créer une figure et un ensemble de sous-parcelles
sns.histplot(df_eco['Année Plus Récente'], kde = True, color='SteelBlue') #Dessine un histogramme à partir des colonnes du DataFrame
plt.title('Indicateur économique')
plt.xlabel('NY.GNP.PCAP.PP.CD')

# ------------------------------------------------------------------------------------
# Indicateur éducatif lycée : SE.SEC.ENRR
# ------------------------------------------------------------------------------------
df_edu_sec=df_Edstat_Total_2015[df_Edstat_Total_2015['Indicator Code'] == 'SE.SEC.ENRR']

plt.subplot(5,2,5) #Créer une figure et un ensemble de sous-parcelles
sns.boxplot( x=df_edu_sec['Indicator Code'], y=df_edu_sec['Année Plus Récente'], width=0.5, color='SteelBlue') #Crée une figure à moustaches à partir des colonnes DataFrame
plt.title('Indicateur éducatif lycée')
plt.ylabel('Taux scolarisé lycée (%)')

plt.subplot(5,2,6) #Créer une figure et un ensemble de sous-parcelles
sns.histplot(df_edu_sec['Année Plus Récente'], kde = True, color='SteelBlue') #Dessine un histogramme à partir des colonnes du DataFrame
plt.title('Indicateur éducatif lycée')
plt.xlabel('SE.SEC.ENRR')

# ------------------------------------------------------------------------------------
# Indicateur éducatif étudiant : SE.TER.ENRR
# ------------------------------------------------------------------------------------
df_edu_ter=df_Edstat_Total_2015[df_Edstat_Total_2015['Indicator Code'] == 'SE.TER.ENRR']

plt.subplot(5,2,7) #Créer une figure et un ensemble de sous-parcelles
sns.boxplot( x=df_edu_ter['Indicator Code'], y=df_edu_ter['Année Plus Récente'], width=0.5, color='SteelBlue')#Crée une figure à moustaches à partir des colonnes DataFrame
plt.title('Indicateur éducatif ens. supérieur')
plt.ylabel('Taux scolarisé ens. supérieur (%)')

plt.subplot(5,2,8) #Créer une figure et un ensemble de sous-parcelles
sns.histplot(df_edu_ter['Année Plus Récente'], kde = True, color='SteelBlue') #Dessine un histogramme à partir des colonnes du DataFrame
plt.title('Indicateur éducatif ens. supérieur')
plt.xlabel('SE.TER.ENRR')

# ------------------------------------------------------------------------------------
# Indicateur numérique : SE.TER.ENRR
# ------------------------------------------------------------------------------------
df_num=df_Edstat_Total_2015[df_Edstat_Total_2015['Indicator Code'] == 'IT.NET.USER.P2']

plt.subplot(5,2,9) #Créer une figure et un ensemble de sous-parcelles
sns.boxplot( x=df_num['Indicator Code'], y=df_num['Année Plus Récente'], width=0.5, color='SteelBlue')#Crée une figure à moustaches à partir des colonnes DataFrame
plt.title('Indicateur numérique')
plt.ylabel('Taux d\'tilisateur d\'internet (%)')

plt.subplot(5,2,10) #Créer une figure et un ensemble de sous-parcelles
sns.histplot(df_num['Année Plus Récente'], kde = True, color='SteelBlue') #Dessine un histogramme à partir des colonnes du DataFrame
plt.title('Indicateur numérique')
plt.xlabel('IT.NET.USER.P2')
plt.savefig("pie_char_statitisque")
plt.tight_layout()

*Notes :*
***
- L'indicateur numérique présente une forme bimodale avec 2 pics qui indique qu'il y a deux groupes différents(à 20% et à 60%).
- Le taux de scolarisation dans l'enseignement supérieur est plus ou moins reparti entre les pays que le taux de scolarisation au niveau lycée.
- Présence d'outliers pour les indicateurs démographique, économique et éducatif au niveau lycée.   

In [ ]:
# Statistiques descriptives

#moyenne de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
mean_num=df_num['Année Plus Récente'].mean()
mean_eco=df_eco['Année Plus Récente'].mean()
mean_edu_sec=df_edu_sec['Année Plus Récente'].mean()
mean_edu_ter=df_edu_ter['Année Plus Récente'].mean()
mean_pop=df_pop['Année Plus Récente'].mean()

#médiane de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
median_num=df_num['Année Plus Récente'].median()
median_eco=df_eco['Année Plus Récente'].median()
median_edu_sec=df_edu_sec['Année Plus Récente'].median()
median_edu_ter=df_edu_ter['Année Plus Récente'].median()
median_pop=df_pop['Année Plus Récente'].median()

#variance de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
var_num=df_num['Année Plus Récente'].var(ddof=0)
var_eco=df_eco['Année Plus Récente'].var(ddof=0)
var_edu_sec=df_edu_sec['Année Plus Récente'].var(ddof=0)
var_edu_ter=df_edu_ter['Année Plus Récente'].var(ddof=0)
var_pop=df_pop['Année Plus Récente'].var(ddof=0)

#l'écart type de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
std_num=df_num['Année Plus Récente'].std(ddof=0)
std_eco=df_eco['Année Plus Récente'].std(ddof=0)
std_edu_sec=df_edu_sec['Année Plus Récente'].std(ddof=0)
std_edu_ter=df_edu_ter['Année Plus Récente'].std(ddof=0)
std_pop=df_pop['Année Plus Récente'].std(ddof=0)

#l'asymétrie non biaisée de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
skew_num=df_num['Année Plus Récente'].skew()
skew_eco=df_eco['Année Plus Récente'].skew()
skew_edu_sec=df_edu_sec['Année Plus Récente'].skew()
skew_edu_ter=df_edu_ter['Année Plus Récente'].skew()
skew_pop=df_pop['Année Plus Récente'].skew()

#l'aplatissement non biaisé de tous les indicateurs "numérique", "économique","population",'education secondaire'"éducation tertiaire"
kurtosis_num=df_num['Année Plus Récente'].kurtosis()
kurtosis_eco=df_eco['Année Plus Récente'].kurtosis()
kurtosis_edu_sec=df_edu_sec['Année Plus Récente'].kurtosis()
kurtosis_edu_ter=df_edu_ter['Année Plus Récente'].kurtosis()
kurtosis_pop=df_pop['Année Plus Récente'].kurtosis()

#Définition d'un dataframe contenant les moyennes, les médianes, les variances,l'asymétrie non biaisée,l'aplatissement non biaisé de tous les indicateurs
data_stats = [['mean',mean_num, mean_eco, mean_edu_sec, mean_edu_ter, mean_pop],
        ['median', median_num, median_eco, median_edu_sec,median_edu_ter,median_pop],
        ['var',var_num, var_eco, var_edu_sec,var_edu_ter,var_pop], 
        ['std',std_num, std_eco,std_edu_sec, std_edu_ter, std_pop], 
        ['skew', skew_num, skew_eco, skew_edu_sec, skew_edu_ter, skew_pop], 
        ['kurtosis', kurtosis_num, kurtosis_eco, kurtosis_edu_sec, kurtosis_edu_ter, kurtosis_pop]]
df_Edstat_Total_stat = pd.DataFrame(data_stats,columns=['Desc','Stat_num','Stat_eco','Stat_edu_sec','Stat_edu_ter','Stat_pop'])
df_Edstat_Total_stat

- Pour l'indicateur démographique et économique : les courbres sont étalées à droite car **le skewness est positif**,elles sont moins aplaties que la distribution normale car le **kurtosis empirique est positif** et **leur variance est élevée**.
- Pour l'indicateur éducatif niveau lycée : la courbe est bimodale et étalée à gauche car le **skewness est négatif**, elles sont plus aplatie que la distribution normale car le **kurtosis empirique est négatif**.
- Pour l'indicateur éducatif de l'enseignement supérieur : la courbe est bimodale et étalée à droite car le **skewness est positif**, elles sont plus aplatie que la distribution normale car le **kurtosis empirique négatif**.  
- Pour l'indicateur numérique : la courbe est bimodale et étalée à gauche car le **skewness est négatif**,ells sont plus aplatie que la distribution normale car le **kurtosis empirique négatif**.

## <span style='Thistle'>5. Score</span>

<span style='PowderBlue'>**Imputation des valeurs manquantes**</span>

In [ ]:
# Utilistion de la methode Interpolat pour remplir les valeurs manquantes du dataframe 
df_Edstat_Totalna=df_Edstat_Total.copy()
plage_temp=df_Edstat_Totalna.columns[4:-2].tolist()
#  Variables quantitatives
df_date_numeric = df_Edstat_Totalna[plage_temp] #dataframe comptant la plage temporelle 2010-2015
# Variables qualitatives
df_objet_categ = df_Edstat_Totalna.loc[:, ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Income Group', 'Region']]
#remplissage des valeurs manquantes du data frame date
df_date_numeric.interpolate(method='linear', axis=1, inplace=True)
df_Edstat_Totalall = pd.concat([df_objet_categ, df_date_numeric], axis=1) #concaténation de deux dataframes
nb_pays=df_Edstat_Totalall['Country Name'].nunique() #affiche le nombre de valeurs unique le colonne country Name
print(f'nombre de pays : {nb_pays}')

<span style='PowderBlue'>**Constitution du dataframe pour établir un score**</span>

In [ ]:
# on ne conserve que les 5 indicateurs avec, pour chaque indicateur et pays, la dernière information réelle ou interpolée de 2015
df_Edstat_Total_score= df_Edstat_Totalall.pivot_table(index=['Country Name'], columns="Indicator Code")['2015']#Crée un tableau croisé dynamique de style feuille de calcul en tant que DataFrame.
df_Edstat_Total_score.shape

<span style='PowderBlue'>**Indicateurs manquants par pays**</span>

In [ ]:
# Affiche un graphe  des données manquantes sur la colonne des indicateurs codes
plt.figure(figsize=(20,10)) 
sns.heatmap(df_Edstat_Total_score.isna(), cbar=False)
plt.savefig("pie_char_indicateu_rmanquant")
plt.show()  

In [ ]:
# Liste des pays avec au moins un indicateur manquant
s_pays_indic_manquant=df_Edstat_Total_score.notnull().sum(axis=1)
i_pays_indic_manquant=s_pays_indic_manquant[s_pays_indic_manquant<5].index
print(f'Nombre de pays à retirer : {len(i_pays_indic_manquant)}')
df_Edstat_Total_retires=df_Edstat_Total_score.loc[i_pays_indic_manquant, :].sort_values(by='SP.POP.1524.TO.UN', ascending=False)
df_Edstat_Total_retires


- **le Canada** est écarté, pénalisé par l'absence du taux de scolarisation dans l'enseignement supérieur alors que les indicateurs numérique, économique et éducatif niveau lycée sont élevés. 

**Conclusion** : **on retire ces pays** avec au moins un indicateur manquant.

In [ ]:
# On ne conserve que les pays avec les 5 indicateurs renseignés
df_Edstat_Total_score.dropna(thresh = 5, inplace=True)  # supprime les lignes contenat des valeurs manquantes
print(f'Nombre de pays restant : {df_Edstat_Total_score.shape[0]}')

<span style='PowderBlue'>**Mise à l'échelle des données**</span>

In [ ]:
cols_indic = ['IT.NET.USER.P2','NY.GNP.PCAP.PP.CD','SE.SEC.ENRR','SE.TER.ENRR','SP.POP.1524.TO.UN']

Les statistiques descriptives des indicateurs ont montré la présence de courbes asymétriques (étalées à droite ou à gauche), les courbes de l'indicateur démographique et économique ont des observations plus concentrées et une variance élevée, avec des outliers. 

**La mise à l'échelle sera effectuée avec PowerTransformer()**.

**Pondération**
                             
| Nature | Nom indicateur | Pondération |
| --- | --- | --- |
| démographique | SP.POP.1524.TO.UN | 4 |
| économique | NY.GNP.PCAP.PP.CD | 1 |
| éducatif | SE.SEC.ENRR | 2 |
| éducatif | SE.TER.ENRR | 3 |
| numérique | IT.NET.USER.P2 | 4 |

On définit une liste de poids à attribuer à chaque indicateur selon ce qui nous a semblé plus pertinent

**Score d'attractivité**

In [ ]:
# Calcul du score pour chaque pays en fonction des pondérations de chaque indicateur
#  num, eco, ed_sec, ed_ter et pop
scaler_pow = PowerTransformer(method = 'box-cox') #Applique une transformation de puissance ne fonctionne qu'avec des valeurs strictement positives
df_Edstat_Total_score_scaled= scaler_pow.fit_transform(df_Edstat_Total_score[cols_indic]) # la méthode fit_transform ajuste aux données, puis les transformer
df_Edstat_Total_score_scaled = pd.DataFrame(df_Edstat_Total_score_scaled, columns=cols_indic)
df_Edstat_Total_score_scaled['Country Name'] = df_Edstat_Total_score.index
df_Edstat_Total_score_scaled['Score']=[(row[0] *4 + row[1] *1 + row[2] *2 + row[3] *3 + row[4]*4) for row in zip(df_Edstat_Total_score_scaled['IT.NET.USER.P2'], df_Edstat_Total_score_scaled['NY.GNP.PCAP.PP.CD'], df_Edstat_Total_score_scaled['SE.SEC.ENRR'], df_Edstat_Total_score_scaled['SE.TER.ENRR'], df_Edstat_Total_score_scaled['SP.POP.1524.TO.UN'])]
df_Edstat_Total_score_scaled.sort_values(by='Score',ascending=False).head(10)

**Score d'attractivité par région**

In [ ]:
# Ajout de la région et du groupe de revenus
region_income = pd.DataFrame({'Country_Name' : df_Edstat_Total['Country Name'], 
                                 'Region' : df_Edstat_Total['Region'], 
                                 'Income Group' : df_Edstat_Total['Income Group']}) #Création d'un dataframe
df_Edstat_Total_score_scaled_region = pd.merge(df_Edstat_Total_score_scaled, region_income, left_on='Country Name', right_on='Country_Name', how='inner') #jointure des dataframes
df_Edstat_Total_score_scaled_region.drop(columns=['Country_Name'],inplace=True)
df_Edstat_Total_score_scaled_region.drop_duplicates(inplace=True) #Effacer les doublons
df_Edstat_Total_score_scaled_region.sort_values(by='Score',ascending=False).head(10);

# Constitution du dataframe de visualisation du regroupement par pays
df_Edstat_Total_score_region = df_Edstat_Total_score_scaled_region.groupby('Region').mean() #on groupe le dataframe en fonction de la moyenne de la région
df_Edstat_Total_score_region=df_Edstat_Total_score_region.rename(columns={'IT.NET.USER.P2':'num',
                                                'NY.GNP.PCAP.PP.CD':'eco',
                                                'SE.SEC.ENRR':'edu_sec',
                                                'SE.TER.ENRR':'edu_ter',
                                                 'SP.POP.1524.TO.UN':'pop'}) #renomme les colonnes du dataframe
df_Edstat_Total_score_region=df_Edstat_Total_score_region.sort_values(by='Score',ascending=False) #trie les valeurs à partir du score

df_Edstat_Total_plot = df_Edstat_Total_score_region[['num','eco','edu_sec','edu_ter','pop']]
plt.rc('axes', facecolor = 'Gainsboro')

# variables quantitatives
categories=list(df_Edstat_Total_plot.columns)

N = len(categories)
 
#  l’angle de chaque axe dans le graphique
angles = [n / float(N) * 2 * pi for n in range(N)]

 
fig = plt.figure(1, figsize=(20,18))
    
colors = ['PowderBlue', 'Coral', 'HotPink', 'LightGreen', 'Cyan', 'RoyalBlue','Peru']

for i, region in enumerate(df_Edstat_Total_plot.index) : 
    
    ax = fig.add_subplot(3,3,i+1, polar=True)

    ax.set_theta_offset(2*pi/3)
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)

    # Tracer une axe par variable 
    plt.xticks(angles, categories, size=8)
    plt.yticks(color="grey", size=2)
    values=df_Edstat_Total_plot.iloc[i].values
    ax.plot(angles, values, 'o-', linewidth=1, linestyle='solid')
    ax.fill(angles, values, colors[i], alpha=0.55)
    ax.set_xticks(angles)
    ax.set_xticklabels(categories, fontsize=12)
    ax.set_title(region, size=20, color=colors[i])
    ax.grid(True)
    plt.grid(True)
    plt.ylim(-2,2)
plt.savefig("pie_char_aire_region") #enregsitre l'image 
plt.show()

Plus l'aire dans le graphique est grande, plus la region a un potentiel élevé.

In [ ]:
#Graphe qui affiche une figure matricielle à 2 dimensions qui donne une visualisation de données numériques sous forme de cellules
fig = plt.figure(1, figsize=(7, 7))
res = sns.heatmap(df_Edstat_Total_score_region, square=True, linewidths=0.1, cmap='coolwarm')
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 18)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 13)
plt.savefig("pie_char_region_par_ordre_tableau") #enregsitre l'image 
plt.show()

On constate que parmis tout les régions , 3 régions se distinguent :

'North America',
'Europe & Central Asia'
et East Asia & Pacific'.

**Score d'attractivité par groupe de revenus**

In [ ]:

 #on groupe le dataframe en fonction de la moyenne du groupe de révenu
df_Edstat_Total_score_income = df_Edstat_Total_score_scaled_region.groupby('Income Group').mean()
df_Edstat_Total_score_income = df_Edstat_Total_score_income.rename(columns={'IT.NET.USER.P2':'num',
                                                'NY.GNP.PCAP.PP.CD':'eco',
                                                'SE.SEC.ENRR':'edu_sec',
                                                'SE.TER.ENRR':'edu_ter',
                                                 'SP.POP.1524.TO.UN':'pop'})

df_Edstat_Total_score_income=df_Edstat_Total_score_income.sort_values(by='Score', ascending=False)

df_Edstat_Total_plot = df_Edstat_Total_score_income[['num','eco','edu_sec','edu_ter','pop']]

plt.rc('axes', facecolor = 'Gainsboro')

# variable quantitatives
categories=list(df_Edstat_Total_plot.columns)
N = len(categories)
 
#  l’angle de chaque axe dans le graphique
angles = [n / float(N) * 2 * pi for n in range(N)]

 
fig = plt.figure(1, figsize=(20,13))
    
colors = ['SandyBrown', 'MediumPurple', 'MediumSeaGreen', 'LightGreen', 'FireBrick']

for i, region in enumerate(df_Edstat_Total_plot.index) : 
    
    ax = fig.add_subplot(2,3,i+1, polar=True)

    ax.set_theta_offset(2*pi/3)
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)

    #  # Tracer une axe par variable
    plt.xticks(angles, categories, size=16)
    plt.yticks(color="grey", size=2)
    values=df_Edstat_Total_plot.iloc[i].values
    ax.plot(angles, values, 'o-', linewidth=1, linestyle='solid')
    ax.fill(angles, values, colors[i], alpha=0.55)
    ax.set_xticks(angles)
    ax.set_xticklabels(categories, fontsize=12)
    ax.set_title(region, size=20, color=colors[i])
    ax.grid(True)
    plt.grid(True)
    plt.ylim(-2,2)
plt.savefig("pie_char_aire_groupe_revenu") #enregsitre l'image 
plt.show()

Plus l'aire dans le graphique est grande, plus le groupe de revenu a du potentiel.

In [ ]:
#Graphe qui affiche une figure matricielle à 2 dimensions qui donne une visualisation de données numériques sous forme de cellules
fig = plt.figure(1, figsize=(7, 7))
ax = sns.heatmap(df_Edstat_Total_score_income, square=True, linewidths=0.1, cmap='coolwarm')
ax.tick_params(axis='x',labelsize=13)
ax.tick_params(axis='y',labelsize=18)
plt.savefig("pie_char_groupe_revenu_par_ordre") #enregsitre l'image 
plt.show()

Les groupes de revenus qui qe distingue sont:
- High income OECD et non OECD.
- Upper middle income

<span style='PowderBlue'>**Score d'attractivité représenté sur une carte du monde**</span>

In [ ]:
#Importation du dataset des pays du monde de geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) 

# Dataset geopandas : code ISO-3166 alpha 3 et Nom court
df_Edstat_Total1=world.sort_values(by = ['iso_a3'])
df_Edstat_Total1=df_Edstat_Total1.loc[:,['iso_a3','name']]
df_Edstat_Total1=df_Edstat_Total1.rename(columns={'iso_a3':'ISO'})
df_Edstat_Total1=df_Edstat_Total1.set_index('ISO')

#  Dataset EdStatsCountry.csv : code ISO-3166 alpha 3 et Nom court
df_Edstat_Total2 = EdStatsCountry.sort_values(by = ['Country Code'])
df_Edstat_Total2 = df_Edstat_Total2.loc[:,['Country Code','Short Name']]
df_Edstat_Total2 = df_Edstat_Total2.rename(columns={'Country Code':'ISO'})
df_Edstat_Total2 = df_Edstat_Total2.set_index('ISO')

# Fusion des 2 dataframes
df_Edstat_Total_comp_nom_pays= pd.merge(df_Edstat_Total1, df_Edstat_Total2, on='ISO', how='left')
df_Edstat_Total_comp_nom_pays.reset_index(inplace=True)

# Recherche des noms de pays différents entre geopandas et notre jeu de données
rows = df_Edstat_Total_comp_nom_pays[df_Edstat_Total_comp_nom_pays['name'] != df_Edstat_Total_comp_nom_pays['Short Name']]
rows.drop([0,1,2,3,4], inplace=True)
df_Edstat_Total_comp=rows[rows['Short Name'].notnull()].sort_values(by='ISO')
noms_a_changer=df_Edstat_Total_comp['Short Name'].tolist()
codes_iso=df_Edstat_Total_comp['ISO'].tolist()

# Récupération des index geopandas des noms de pays à changer
index_geo=world[world['iso_a3'].isin(codes_iso)].index

# Récupération des codes ISO 3 de geopandas
codes_isoa3=world.loc[index_geo,'iso_a3'].tolist()

# Parcours les codes ISO 3 pour avoir une liste de noms de pays dans l'ordre des index geopandas
noms_a_changer=[]
[noms_a_changer.append(df_Edstat_Total_comp[df_Edstat_Total_comp['ISO'] == iso]['Short Name'].tolist()[0]) for iso in codes_isoa3]
# Changement du libellé des noms courts des pays de geopandas par ceux de notre jeux de données
world.loc[world.index[index_geo.tolist()],'name']=noms_a_changer

# Pays écartés représentés en blanc sur la carte
df_Edstat_Total_pays_ecartes=pd.DataFrame({'Country Name': df_Edstat_Total_retires.index, 'Score': np.zeros((66,), dtype=int)})
df_Edstat_Total_pays_ecartes.drop(index = [20,33,36,37,38,39,40,41,42,43,44,45,46,48,49,50,52,53,55,56,57,59,61,63,65], inplace = True) 
df_Edstat_Total_pays_ecartes.loc[3, 'Country Name'] = 'Venezuela'
df_Edstat_Total_pays_ecartes.loc[34, 'Country Name'] = 'The Bahamas'
df_Edstat_Total_pays_ecartes.loc[6, 'Country Name'] = 'Dem. People\'s Rep. Korea'
df_Edstat_Total_pays_ecartes.loc[47, 'Country Name'] = 'Dominican Republic'
df_Edstat_Total_pays_ecartes.shape

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
df_Edstat_Total_score_geo = world.merge(df_Edstat_Total_score_scaled, left_on = 'name', right_on = 'Country Name')[['geometry','Country Name','Score']]
df_Edstat_Total_pays_ecartes_geo = world.merge(df_Edstat_Total_pays_ecartes, left_on = 'name', right_on = 'Country Name')[['geometry','Country Name','Score']]

fig, ax = plt.subplots()
fig.set_size_inches(30,18)

ax.set_facecolor('xkcd:white')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.1)
df_Edstat_Total_score_geo.plot(column=df_Edstat_Total_score_geo['Score'], legend = True, ax=ax, cax=cax, cmap='RdYlGn',linestyle=":",edgecolor='black' )
df_Edstat_Total_pays_ecartes_geo.plot(column=df_Edstat_Total_pays_ecartes['Score'], legend = True, ax=ax, cax=cax, color='white',linestyle=":",edgecolor='black' )
plt.savefig("pie_char_carte_geographique") #enregsitre l'image 
plt.show()


**Bilan : Top 10 des pays à fort potentiels**

In [ ]:
# on détermine le top 10 des pays à fort potentiels
df_Edstat_Total_top10_prov = df_Edstat_Total_score_scaled[['Country Name','Score']].sort_values(by='Score',ascending=False).head(10)
df_Edstat_Total_top10_prov['Place Provisoire']=np.arange(1, 11, 1) # on cree la colonne place provisoire
df_Edstat_Total_top10_prov.rename(columns={'Country Name': 'Pays'}, inplace=True) #on renomme la colonne country name
df_Edstat_Total_top10_prov.set_index('Place Provisoire') #on défini l'index du dataFrame à l'aide des colonnes existantes.


In [ ]:
# on détermine les régions et groupes de revenus à fort potentiels
df_Edstat_Total_top10_region=df_Edstat_Total[df_Edstat_Total['Country Code'].isin(['AUS', 'USA','GBR','JAP','KOR','BEL','ESP','NDL','DEU','RUS'])][['Country Name','Region','Income Group']]
df_Edstat_Total_top10_region.drop_duplicates(inplace=True)
df_Edstat_Total_top10_region.set_index('Country Name', inplace=True)
df_Edstat_Total_top10_region.sort_values(by='Region', inplace=True)
df_Edstat_Total_top10_region

Les 10 pays les plus attractifs font bien parties des 3 régions :
- East Asia & Pacific,
- Europe & Central Asia,
- North America	

et des 2 groupes de revenus :
- High income: OECD,
- High income: nonOECD


## <span style='Thistle'>5. Conclusion</span>

L’analyse exploratoire des données sur l’éducation de la banque mondiale permettent d’informer le projet d’expansion. Nous avons enregistré le top 10 des pays à fort potentiels tels que United States, Australia, United Kingdom, Japan, Korea, Belgium, Spain, Netherlands, Germany, Russian Federation où l'entreprise peut prendre des décisions d'ouverture. On constate qu'ils font partis des pays les plus émergents dans le monde. 
En effet l'anlyse par région concorde les faits car l'Amérique du Nord ainsi que l'Europe et l'Asie centrale se distinguent comme étant les régions ayant les plus grands scores et également les plus grands potentiels. 
En plus, ces résultats sont assez réelles car ces pays ont avec des économies solides et donc avec un niveau socio-économique permettant de ce fait un niveau d'éducation de la population parmi les plus élevés.
